# data 리키지를 막기 위한 real_test 분리


In [ ]:
import pandas as pd

user = pd.read_csv("/content/drive/Shareddrives/빅콘테스트/데이터/2022빅콘테스트_데이터분석리그_데이터분석분야_퓨처스부문_데이터셋_220908.zip (Unzipped Files)/user_spec.csv")
log = pd.read_csv("/content/drive/Shareddrives/빅콘테스트/데이터/2022빅콘테스트_데이터분석리그_데이터분석분야_퓨처스부문_데이터셋_220908.zip (Unzipped Files)/log_data.csv")
result = pd.read_csv("/content/drive/Shareddrives/빅콘테스트/데이터/2022빅콘테스트_데이터분석리그_데이터분석분야_퓨처스부문_데이터셋_220908.zip (Unzipped Files)/loan_result.csv")

In [ ]:
a = list(result.loc[result['is_applied'].isna(),'application_id'].unique())

In [ ]:
real_test_user = user.loc[user['application_id'].isin(a),]
real_test_result = result.loc[result['is_applied'].isna(),]

In [ ]:
user = user.loc[~(user['application_id'].isin(a))]
result = result[result['is_applied'].notna()]

# user 전처리

## 결측치

In [ ]:
user['credit_score'] = user['credit_score'].fillna(value=0)
user.isna().sum()

application_id                               0
user_id                                      0
birth_year                               10358
gender                                   10358
insert_time                                  0
credit_score                                 0
yearly_income                               89
income_type                                 85
company_enter_month                     147858
employment_type                             85
houseown_type                               85
desired_amount                              85
purpose                                     85
personal_rehabilitation_yn              586351
personal_rehabilitation_complete_yn    1015579
existing_loan_cnt                       168034
existing_loan_amt                       262852
dtype: int64

In [ ]:
#birth year nan 및 gender 관련해서 : 초기 가입자
user['age'] = 2022 - user['birth_year'] + 1
user['age'] = user['age'].fillna(value=user['age'].median())

In [ ]:
user.drop('birth_year', axis=1, inplace=True)

In [ ]:
user['personal_rehabilitation_yn'] = user['personal_rehabilitation_yn'].fillna(value=0)
user['personal_rehabilitation_complete_yn'] = user['personal_rehabilitation_complete_yn'].fillna(value=0)

In [ ]:
user['existing_loan_amt'] = user['existing_loan_amt'].fillna(value=0)
user['existing_loan_cnt'] = user['existing_loan_cnt'].fillna(value=0)

In [ ]:
from tqdm import tqdm 

columns = ['yearly_income','income_type','employment_type','houseown_type','desired_amount','purpose']
a = user.loc[user['yearly_income'].isna()]
for i, row in tqdm(a.iterrows(), total = a.shape[0]):
  b = user.loc[user['user_id']==row['user_id']]
  if len(b) == 1: pass
  else:
    for column in columns:
      try:
        user.at[i,column] = b[column].mode().mean()
      except:
        user.at[i,column] = b[column].mode()[0]

100%|██████████| 89/89 [00:00<00:00, 139.34it/s]


In [ ]:
from tqdm import tqdm 

column = 'gender'
a = user.loc[user['gender'].isna()]
for i, row in tqdm(a.iterrows(), total = a.shape[0]):
  b = user.loc[user['user_id']==row['user_id']]
  if len(b) == 1: pass
  else:
    try:
      user.at[i,column] = b[column].mode()
    except:
      pass




100%|██████████| 10358/10358 [00:35<00:00, 291.61it/s]


In [ ]:
user['yearly_income'].fillna(user['yearly_income'].median(), inplace=True)
user['income_type'].fillna(-1, inplace=True)
user['employment_type'].fillna(-1, inplace=True)
user['houseown_type'].fillna(-1, inplace=True)
user['desired_amount'].fillna(user['desired_amount'].median(), inplace=True)
user['purpose'].fillna(-1, inplace=True)

In [ ]:
# desired_amount를 높은 그룹 낮은 그룹 해서 각각 그룹에서 7:3 해서 남성: 여성
user.loc[user[(user['existing_loan_amt']>30000000)&(user['gender'].isna())][:1157].index, 'gender'] = 1
user.loc[user[(user['existing_loan_amt']>30000000)&(user['gender'].isna())].index, 'gender'] = 0

user.loc[user[(user['existing_loan_amt']<=30000000)&(user['gender'].isna())][:4252].index, 'gender']=1
user.loc[user[(user['existing_loan_amt']<=30000000)&(user['gender'].isna())].index, 'gender']=0

In [ ]:
user.isna().sum()

application_id                              0
user_id                                     0
gender                                      0
insert_time                                 0
credit_score                                0
yearly_income                               0
income_type                                 0
company_enter_month                    147858
employment_type                             0
houseown_type                               0
desired_amount                              0
purpose                                     0
personal_rehabilitation_yn                  0
personal_rehabilitation_complete_yn         0
existing_loan_cnt                           0
existing_loan_amt                           0
age                                         0
dtype: int64

## 카테고리

In [ ]:
user['foreign'] = [0] * len(user)
user.loc[user['purpose'].isin(['LIVING ','SWITCHLOAN','BUSINESS','ETC','HOUSEDEPOSIT','BUYHOUSE','INVEST','BUYCAR']),'foreign'] = 1

In [ ]:
user.loc[user['purpose']=='LIVING','purpose'] = '생활비'
user.loc[user['purpose']=='SWITCHLOAN','purpose'] = '대환대출'
user.loc[user['purpose']=='BUSINESS','purpose'] = '사업자금'
user.loc[user['purpose']=='ETC','purpose'] = '기타'
user.loc[user['purpose']=='HOUSEDEPOSIT','purpose'] = '전월세보증금'
user.loc[user['purpose']=='BUYHOUSE','purpose'] = '주택구입'
user.loc[user['purpose']=='INVEST','purpose'] = '투자'
user.loc[user['purpose']=='BUYCAR','purpose'] = '자동차구입'
user['purpose'].value_counts()

생활비       738643
대환대출      289720
사업자금       51640
기타         25243
전월세보증금     22077
주택구입       13608
투자          9937
자동차구입       3368
-1            31
Name: purpose, dtype: int64

## 추가 처리및 파생변수 
- company_enter_month 결측치 
- 개인회생자 3 카테고리화
- 연수입/ 기대수입

In [ ]:
company_enter_month = '000000'
user.fillna({'company_enter_month':company_enter_month},inplace=True )

In [ ]:
user['company_enter_month']= user['company_enter_month'].astype(str)

user['enter_y'] = user['company_enter_month'].str[:4]  #입사 년
user['enter_m'] = user['company_enter_month'].str[4:6] #입사 월
user['enter_y'] = user['enter_y'].astype(int)
user['enter_m'] = user['enter_m'].astype(int)

In [ ]:
user['insert_time'] = pd.to_datetime(user['insert_time'])
user['insert_time'] = user['insert_time'].dt.to_period(freq='D')
user['insert_time']  = user['insert_time'].values.astype('datetime64[D]')

In [ ]:
def f3(user2):
    if (user2['enter_y'] != 0):
      user2['company_enter_month']= user2['company_enter_month'][:4] + '-' + user2['company_enter_month'][4:6]  + '-'+ '01'
      user2['company_enter_month'] = pd.to_datetime(user2['company_enter_month'])
      diff = user2['insert_time'] - user2['company_enter_month']
      val = diff.days
    else:
        val = 0
    return val

In [ ]:
import numpy as np
user['employee_d'] = user.apply(f3, axis=1)
user['employee_d'] = np.abs(user['employee_d'].astype(int))  #음수로 맘대로 쓴 사람 2명 있어서 이건 그냥 절댓값 씌워버림

In [ ]:
user['employee_y'] = user['employee_d']//365  #근무년수
user['employee_m'] = user['employee_d']//30  #근무개월수 

In [ ]:
user['month'] = user['insert_time'].dt.month #월
user['day'] = user['insert_time'].dt.day #일
user['hour'] = user['insert_time'].dt.hour #시
user['weekday'] = user['insert_time'].dt.weekday # 요일숫자(0-월, 1-화) 
user['week_num'] = user['insert_time'].dt.isocalendar().week  #주차 이름

In [ ]:
#아예 회생자가 아닌 경우
user['personal_rehabilitation_type'] = 0
#회생자&반환완료
user.loc[(user['personal_rehabilitation_yn']==1) & (user['personal_rehabilitation_complete_yn']==1), 'personal_rehabilitation_type'] = 1
#회생자&반환못함
user.loc[(user['personal_rehabilitation_yn']==1) & (user['personal_rehabilitation_complete_yn']==0), 'personal_rehabilitation_type'] = 2

In [ ]:
user['desired_amount'].median()

10000000.0

In [ ]:
#  연수입 / 기대금액 변수생성
user.loc[user['desired_amount']==0, 'desired_amount'] = user['desired_amount'].median()
user['amount_income_rate'] = user['yearly_income']/user['desired_amount']

In [ ]:
#user.to_csv("/content/drive/Shareddrives/빅콘테스트/혜림/preprocessed_user.csv", index=False)

# result 전처리

## 결측치

In [ ]:
# 채우기전에 limit nan = 금액한도없음 & rate nan = 금리없음 product_id 특징 만들어주기
no_limit = result[result['loan_limit'].isnull()]['product_id'].unique()
no_limit_list =[]

for i in no_limit:
  if type(result[result['product_id']==i]['loan_limit'].mean()) == float and pd.isna(result[result['product_id']==21]['loan_limit'].mean()):
    no_limit_list.append(i)
no_limit_list

[21, 180, 13, 76, 193, 232, 116, 52, 165, 53]

In [ ]:
# 같은 product id를 가진 limit의 평균으로 채움
for pr in no_limit:
  try:
    x = int(result.loc[result['product_id']==pr, 'loan_limit'].describe()['mean'])
    result.loc[(result['product_id']==pr)&(result['loan_limit'].isnull()), 'loan_limit'] = x
  except:
    result.loc[(result['product_id']==pr)&(result['loan_limit'].isnull()), 'loan_limit']=int(result['loan_limit'].describe()['max'])

In [ ]:
# loan_rate도 마찬가지로
no_rate = result[result['loan_rate'].isnull()]['product_id'].unique()
no_rate_list =[]

for i in no_limit:
  if type(result[result['product_id']==i]['loan_rate'].mean()) == float and pd.isna(result[result['product_id']==21]['loan_rate'].mean()):
    no_rate_list.append(i)

no_rate_list #[21, 180, 13, 76, 193, 232, 116, 52, 165, 53]

[21, 180, 13, 76, 193, 232, 116, 52, 165, 53]

In [ ]:
# 같은 product id를 가진 rate의 평균으로 채움
for pr in no_rate:
  try:
    x = int(result.loc[result['product_id']==pr, 'loan_rate'].describe()['mean'])
    result.loc[(result['product_id']==pr)&(result['loan_rate'].isnull()), 'loan_rate'] = x
  except:
    result.loc[(result['product_id']==pr)&(result['loan_rate'].isnull()), 'loan_rate']=int(result['loan_rate'].describe()['min'])

## merge

In [ ]:
df = pd.merge(result, user,on='application_id', how='left')

In [ ]:
df=df.dropna()

In [ ]:
df['no_limit_product'] = [0] * len(df)
df.loc[(df['product_id'].isin([i for i in no_limit_list])), 'no_limit_product'] = 1

## 평일이면서 9~17시

In [ ]:
def f4(df):
    if (df['weekday'] <=4) & (df['hour'] in (9,17)) :  
        val = 1
    else:
        val = 0
    return val

In [ ]:
df['loanapply_insert_time'] = pd.to_datetime(df['loanapply_insert_time'])
df['hour'] = df['loanapply_insert_time'].dt.hour #시

In [ ]:
#df['week_pop'] = df.apply(f4, axis=1)

df['week_pop'] = [0] * len(df)
df.loc[(df['weekday'] <=4)&(df['hour'].isin([i for i in range(9, 17+1)])), 'week_pop'] = 1

In [ ]:
#log 변환
import numpy as np
df['employee_d'] = np.log1p(df['employee_d'])
df['employee_y'] = np.log1p(df['employee_y'])
df['employee_m'] = np.log1p(df['employee_m'])

## 은행 상품 bin

In [ ]:
def make_bin(df, variable, n):
    
    data = df
    count, bin_dividers = np.histogram(data[variable], bins=n)
    bin_names=[str(i) for i in range(n)]
    data['%s_bin' % variable] = pd.cut(x=data[variable], bins=bin_dividers, labels=bin_names, include_lowest=True)
    data['%s_bin' % variable] = pd.factorize(data['%s_bin' % variable])[0]
    print(data['%s_bin' % variable], '\n\n')
    
    return data

In [ ]:
data = df.groupby(by=['product_id']).sum()
data = data[['is_applied']]
data = data.reset_index().rename(columns={"index": "id"})

data = make_bin(data, 'is_applied', n=5)

0      0
1      0
2      0
3      0
4      0
      ..
173    4
174    0
175    0
176    1
177    0
Name: is_applied_bin, Length: 178, dtype: int64 




In [ ]:
type_0 = data[data['is_applied_bin']==0]['product_id'].tolist()
type_1 = data[data['is_applied_bin']==1]['product_id'].tolist()
type_2 = data[data['is_applied_bin']==2]['product_id'].tolist()
type_3 = data[data['is_applied_bin']==3]['product_id'].tolist()
type_4 = data[data['is_applied_bin']==4]['product_id'].tolist()

In [ ]:
df['product_bin'] = len(df) * [0]
df['product_bin'] = np.where(df['product_id'].isin(type_0), 1, df['product_bin'])
df['product_bin'] = np.where(df['product_id'].isin(type_1), 2, df['product_bin'])
df['product_bin'] = np.where(df['product_id'].isin(type_2), 3, df['product_bin'])
df['product_bin'] = np.where(df['product_id'].isin(type_3), 4, df['product_bin'])
df['product_bin'] = np.where(df['product_id'].isin(type_4), 5, df['product_bin'])

In [ ]:
data = df.groupby(by=['bank_id']).sum()
data = data[['is_applied']]
data = data.reset_index().rename(columns={"index": "id"})

data = make_bin(data, 'is_applied', n=5)

0     0
1     1
2     1
3     2
4     1
     ..
57    1
58    1
59    0
60    2
61    1
Name: is_applied_bin, Length: 62, dtype: int64 




In [ ]:
type_0 = data[data['is_applied_bin']==0]['bank_id'].tolist()
type_1 = data[data['is_applied_bin']==1]['bank_id'].tolist()
type_2 = data[data['is_applied_bin']==2]['bank_id'].tolist()
type_3 = data[data['is_applied_bin']==3]['bank_id'].tolist()
type_4 = data[data['is_applied_bin']==4]['bank_id'].tolist()

In [ ]:
df['bank_bin'] = len(df) * [0]
df['bank_bin'] = np.where(df['bank_id'].isin(type_0), 1, df['bank_bin'])
df['bank_bin'] = np.where(df['bank_id'].isin(type_1), 2, df['bank_bin'])
df['bank_bin'] = np.where(df['bank_id'].isin(type_2), 3, df['bank_bin'])
df['bank_bin'] = np.where(df['bank_id'].isin(type_3), 4, df['bank_bin'])
df['bank_bin'] = np.where(df['bank_id'].isin(type_4), 5, df['bank_bin'])

# 파생변수

- 기대금액이 충분한지(is_enough)
- 이자액은 얼마인지(interest)
- 신용등급(credit_grade)
- 은행 상품 몇 개 봤는지(viewed_product)
- 신용등급 바뀌는 사람인지(credit_changed)

In [ ]:
df['is_enough'] = (df['desired_amount']-df['loan_limit'])/df['loan_limit'] #inf 가 4969개
df.loc[df['loan_limit']==0, 'is_enough'] = -1

In [ ]:
df['interest'] = df['desired_amount'] * df['loan_rate']

In [ ]:
def grade(x):
 if x>941 : 
   return 1
 elif x>890:
   return 2
 elif x>831:
   return 3
 elif x>767:
   return 4
 elif x>697:
   return 5
 elif x>629:
   return 6
 elif x>529:
   return 7
 elif x>453:
   return 8
 elif x>334:
   return 9
 else:
   return 10

df['credit_grade'] = df['credit_score'].apply(grade) 

In [ ]:
temp = df['application_id'].value_counts()
df['viewd_product'] = df['application_id'].map(temp) 

In [ ]:
a = df.groupby(['user_id'])
b = a['credit_grade'].nunique()!=1

temp = b.reset_index()
temp.loc[temp['credit_grade']==True, 'user_id']

df['credit_changed'] = [0] * len(df)
df.loc[df['user_id'].isin(temp.loc[temp['credit_grade']==True, 'user_id'].tolist()), 'credit_changed'] = 1 

In [ ]:
#몇 번째 신청인지
temp = df.groupby(['user_id','application_id'])['loanapply_insert_time'].rank(method='min', ascending=True)
df['appli_order'] = temp

#각 application_id 마다 loan_limit 기준으로 순위
temp = df.groupby('application_id')['loan_limit'].rank(method='min', ascending=False)
df['limit_order'] = temp

# 지금 받는 월급으로 몇 년을 갚아야 할지 / 근데 이건 중요 변수 아니긴 했음
df['loan_year'] = df['desired_amount']/df['yearly_income']
df.replace([np.inf, -np.inf],100, inplace=True)

In [ ]:
#희망금액이 연소득보다 많은가 ? => over_desire 욕심쟁이
df['over_desire'] = (df['desired_amount']-df['yearly_income'])/df['yearly_income'] #inf인거 한 6000개 희망금액이랑 연소득 비교해서 희망금액이 높으면 over_desrie다 
df.loc[df['yearly_income']==0, 'over_desire'] = 1

#희망금액이 기존의 대출금액보다 많은가 ?  => over_desrie2
df['over_desire2'] = (df['desired_amount']-df['existing_loan_amt'])/df['existing_loan_amt'] #inf인거 한 40000개
df.loc[df['existing_loan_amt']==0, 'over_desire2'] = 1

In [ ]:
#기존대출경험 있는 사람 => experience
def experience(x):
 if x>=1 : 
   return 1
 else:
   return 0

df['experience'] = df['existing_loan_cnt'].apply(experience) 

In [ ]:
#'personal_rehabilitation_yn',personal_rehabilitation_complete_yn','enter_m', 'employee_y', 'employee_m''month', 'day' 미리 삭제
df = df[['application_id', 'loanapply_insert_time', 'bank_id', 'product_id','loan_limit', 'loan_rate', 'is_applied', 'user_id', 'gender', 'credit_score', 'yearly_income', 'income_type',
        'employment_type', 'houseown_type', 'desired_amount', 'purpose', 'existing_loan_cnt','existing_loan_amt', 'age', 'foreign', 'enter_y','employee_d', 'hour','weekday', 'week_num', 'personal_rehabilitation_type',
       'amount_income_rate', 'week_pop', 'product_bin', 'bank_bin', 'is_enough', 'interest', 'credit_grade', 'viewd_product','appli_order', 'limit_order', 'loan_year',
       'over_desire', 'over_desire2']]

In [ ]:
df.to_csv("/preprocessed_df2.csv", index=False)

In [ ]:
import pandas as pd
df = pd.read_csv("/preprocessed_df2.csv")

In [ ]:
data1 = pd.read_csv("/복수통계_07172540.csv")
data2 = pd.read_csv("/소비자동향조사(지역, 월)_07172513.csv")

df['loanapply_insert_time'] = pd.to_datetime(df['loanapply_insert_time'])
df['loanapply_insert_time']  = df['loanapply_insert_time'].values.astype('datetime64[D]')

In [ ]:
data1['변환'] = pd.to_datetime(data1['변환'])
data1['변환'] = data1['변환'].values.astype('datetime64[D]')
data1['변환']
data1 = data1.fillna(method='ffill')

In [ ]:
data1.columns = ['loanapply_insert_time','news','interest_rate']
df = pd.merge(df, data1[['loanapply_insert_time','news']],on='loanapply_insert_time', how='left')

In [ ]:
df = df.sort_values(by=['loanapply_insert_time'])
df['news'] = df['news'].fillna(method='bfill')

In [ ]:
#상환능력
df['ablilty'] = df['existing_loan_amt']/df['yearly_income']*100
df.loc[df['yearly_income']==0, 'ablilty'] = 0

#상환의지


#총부채원리금상환비율(DSR) : 원리금 나누기 연소득
df['dsr'] = df['loan_limit']/df['yearly_income']*100
df.loc[df['yearly_income']==0, 'dsr'] = 0

In [ ]:
a = df.groupby(['user_id'])
b = a['yearly_income'].nunique()>=2

temp = b.reset_index()
temp.loc[temp['yearly_income']==True, 'user_id']

df['income_changed'] = [0] * len(df)
df.loc[df['user_id'].isin(temp.loc[temp['yearly_income']==True, 'user_id'].tolist()), 'income_changed'] = 1   #변동o : 400만, 변동x : 600만

In [ ]:
df[df['income_changed']==0]

# 저장

# split

In [ ]:
# train:5월 그 전 + 100만개 train,val은(train에 34월 val에 5월12주) / test:5월 마지막 둘째주
## 3,4월로 5월 예측
train = df[df['week_num'] < 20]
test = df[(df['week_num'] >=20)] #5월 16~31일

In [ ]:
#len(train[train['is_applied']==0])

In [ ]:
app1 = train[(train['is_applied']==1) & (train['income_changed']==0)].sample(n=200000, replace=False)  #1:9, 2:8, 3:7(0.22), 4:6
app0 = train[(train['is_applied']==0) & (train['income_changed']==0)].sample(n=2000000, replace=False)  # 20만 : 200만 = 0.23 / 20:180만 = 0.22  30만 : 70만 = 0.22/ 20만:400만 :0.20
train = pd.concat([app1,app0], axis = 0).reset_index(drop=True)

In [ ]:
train

In [ ]:
train.to_csv("/content/drive/Shareddrives/빅콘테스트/221009/1009_train.csv", index=False)
test.to_csv("/content/drive/Shareddrives/빅콘테스트/221009/1009_test.csv", index=False)